In [13]:
import sys
import os

path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path)

In [14]:
from diffusion.data_loaders.spinkick_diffuser_dataset import SpinkickDiffuserDataset
dataset = SpinkickDiffuserDataset("/home/kenji/Fyp/DeepMimic_mujoco/diffusion/data/motions/humanoid3d_spinkick.txt")
len(dataset), dataset[0], dataset[0].trajectories.shape

(100,
 Batch(trajectories=tensor([[ 0.0000,  0.0000,  0.8251,  ..., -0.5877, -0.5578, -0.1123],
         [-0.0098, -0.0053,  0.8223,  ..., -0.6034, -0.5496, -0.1377],
         [-0.0205, -0.0135,  0.8187,  ..., -0.6073, -0.5372, -0.1672],
         ...,
         [ 0.2570, -0.0458,  0.8492,  ..., -0.3039, -0.5771, -0.0356],
         [ 0.2659, -0.0516,  0.8421,  ..., -0.2956, -0.5725, -0.0793],
         [ 0.2787, -0.0612,  0.8371,  ..., -0.3081, -0.5633, -0.1231]]), conditions={0: tensor([ 0.0000,  0.0000,  0.8251, -0.9969,  0.0457, -0.0639, -0.0029, -0.0399,
          0.0621, -0.0921,  0.0930,  0.4230,  0.1155, -1.4425,  0.4709, -0.4550,
          1.1228,  1.3071, -0.7625, -0.2475,  1.4452, -0.1059, -0.4803,  0.0466,
         -0.8114,  0.0888, -0.3455, -0.1734,  0.2040, -0.1554,  0.3841, -0.6268,
         -0.5877, -0.5578, -0.1123])}),
 torch.Size([64, 35]))

### Model NN

In [15]:
import torch
from diffusion.diffuser.utils import Trainer as dTrainer, Config as dConfig

savepath = '/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser'
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [16]:
from diffusion.diffuser.models.temporal import TemporalUnet 

horizon = dataset[0].trajectories.shape[0]
horizon=64
transition_dim = dataset[0].trajectories.shape[1]

model_config = dConfig(
    TemporalUnet,
    savepath=(savepath, 'model_config.pkl'),
    horizon=horizon,
    transition_dim=transition_dim,
    cond_dim=transition_dim,
    device=device,
)
model = model_config()


[utils/config ] Config: <class 'diffusion.diffuser.models.temporal.TemporalUnet'>
    cond_dim: 35
    horizon: 64
    transition_dim: 35

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser/model_config.pkl

[ models/temporal ] Channel dimensions: [(35, 32), (32, 64), (64, 128), (128, 256)]
[(35, 32), (32, 64), (64, 128), (128, 256)]


In [17]:
from diffusion.diffuser.models.diffusion import GaussianDiffusion

n_timesteps = 1000
loss_type = 'l2'
clip_denoised = False
predict_epsilon = False
action_weight = 5
loss_weights = None
loss_discount = 1

diffusion_config = dConfig(
    GaussianDiffusion,
    savepath=(savepath, 'diffusion_config.pkl'),
    horizon=horizon,
    transition_dim=transition_dim,
    n_timesteps=n_timesteps,
    loss_type=loss_type,
    clip_denoised=clip_denoised,
    predict_epsilon=predict_epsilon,
    # loss weighting
    action_weight=action_weight,
    loss_weights=loss_weights,
    loss_discount=loss_discount,
    device=device,
)

diffusion = diffusion_config(model)


[utils/config ] Config: <class 'diffusion.diffuser.models.diffusion.GaussianDiffusion'>
    action_weight: 5
    clip_denoised: False
    horizon: 64
    loss_discount: 1
    loss_type: l2
    loss_weights: None
    n_timesteps: 1000
    predict_epsilon: False
    transition_dim: 35

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser/diffusion_config.pkl



In [18]:
learning_rate = 2e-4
gradient_accumulate_every = 2
ema_decay = 0.995
sample_freq = 2000
save_freq = 2000
n_train_steps = 1e5
n_saves = 5
save_parallel = False
bucket = None
n_reference = 8
train_batch_size = 32

trainer_config = dConfig(
    dTrainer,
    savepath=(savepath, 'trainer_config.pkl'),
    train_batch_size=train_batch_size,
    train_lr=learning_rate,
    gradient_accumulate_every=gradient_accumulate_every,
    ema_decay=ema_decay,
    sample_freq=sample_freq,
    save_freq=save_freq,
    label_freq=int(n_train_steps // n_saves),
    save_parallel=save_parallel,
    results_folder=savepath,
    bucket=bucket,
    n_reference=n_reference,
)

trainer = trainer_config(diffusion, dataset, renderer=None)


[utils/config ] Config: <class 'diffusion.diffuser.utils.training.Trainer'>
    bucket: None
    ema_decay: 0.995
    gradient_accumulate_every: 2
    label_freq: 20000
    n_reference: 8
    results_folder: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser
    sample_freq: 2000
    save_freq: 2000
    save_parallel: False
    train_batch_size: 32
    train_lr: 0.0002

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser/trainer_config.pkl



In [19]:
import torch
from diffuser.utils import batchify

tunet = model
tunet.to(device)
test_data = dataset[0]
batch = batchify(test_data)
t = torch.randint(0, 1000, (1,), device=device).long().to(device)
res = tunet(batch.trajectories, cond=batch.conditions, time=t, verbose=True)
res, res.shape

x.shape torch.Size([1, 64, 35])
x.shape torch.Size([1, 35, 64])
x.shape torch.Size([1, 32, 64])
1
xfinal.shape torch.Size([1, 32, 32])
x.shape torch.Size([1, 64, 32])
2
xfinal.shape torch.Size([1, 64, 16])
x.shape torch.Size([1, 128, 16])
3
xfinal.shape torch.Size([1, 128, 8])
x.shape torch.Size([1, 256, 8])
4
xfinal.shape torch.Size([1, 256, 8])
xt1.shape torch.Size([1, 256, 8])
xt2.shape torch.Size([1, 256, 8])
xt3.shape torch.Size([1, 256, 8])
pop.shape torch.Size([1, 256, 8])
pop.shape torch.Size([1, 128, 16])
pop.shape torch.Size([1, 64, 32])


(tensor([[[ 0.0486,  0.0216, -0.2549,  ...,  0.1628, -0.2659,  0.2914],
          [-0.1338,  0.0198, -0.2154,  ...,  0.1263, -0.2677,  0.1393],
          [ 0.0705,  0.5396, -0.3845,  ...,  0.1983, -0.0201,  0.0996],
          ...,
          [-0.1689, -0.0434,  0.1357,  ..., -0.0506, -0.3456,  0.5725],
          [ 0.1514, -0.0490, -0.1641,  ...,  0.1659, -0.1663, -0.1476],
          [-0.1230,  0.0090, -0.0428,  ..., -0.0691, -0.3072,  0.2896]]],
        device='cuda:0', grad_fn=<PermuteBackward0>),
 torch.Size([1, 64, 35]))

In [20]:
from diffuser.utils import report_parameters, batchify

report_parameters(model)

print('Testing forward...', end=' ', flush=True)
x = dataset[0]
batch = batchify(x)
loss, _ = diffusion.loss(batch.trajectories, {})
loss.backward()
print('✓')

[ utils/arrays ] Total parameters: 3.96 M
         downs.3.0.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         ups.0.0.blocks.0.block.0.weight: 327.68 k | Conv1d(512, 128, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.0.b

In [21]:
n_steps_per_epoch = 1000
n_epochs = int(n_train_steps // n_steps_per_epoch)
print(n_epochs)
n_epochs = 1

for i in range(n_epochs):
    print(f'Epoch {i} / {n_epochs} | {savepath}')
    trainer.train(n_train_steps=n_steps_per_epoch)

trainer.save(n_epochs)

100
Epoch 0 / 1 | /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser
[ utils/training ] Saved model to /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser/state_0.pt
0:   0.3724 | a0_loss:   0.0839 | t:   0.2161
100:   0.0843 | a0_loss:   0.0108 | t:   8.9418
200:   0.0388 | a0_loss:   0.0044 | t:   8.1961
300:   0.0207 | a0_loss:   0.0020 | t:   8.1673
400:   0.0122 | a0_loss:   0.0009 | t:   8.3210
500:   0.0074 | a0_loss:   0.0005 | t:   8.4207
600:   0.0044 | a0_loss:   0.0003 | t:   8.4918
700:   0.0027 | a0_loss:   0.0002 | t:   8.4290
800:   0.0018 | a0_loss:   0.0001 | t:   8.4431
900:   0.0013 | a0_loss:   0.0001 | t:   8.6311
[ utils/training ] Saved model to /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser/state_1.pt


In [25]:
# from diffusion.diffuser.utils import load_diffusion

# diffusion_experiment = load_diffusion(
#     "/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser", dataset=dataset, epoch=1)

# renderer = diffusion_experiment.renderer
# model = diffusion_experiment.trainer.ema_model

trainer.load(1)
model = trainer.ema_model

In [36]:
from diffuser.utils import batchify
test = dataset[0]
batch = batchify(test)
sample = diffusion.p_sample_loop(batch.trajectories.shape, batch.conditions)


                                                                                                    
1 / 1000 [                                                            ]   0% | 41.6 Hz
t : 999 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
2 / 1000 [                                                            ]   0% | 54.4 Hz
t : 998 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
3 / 1000 [                                                            ]   0% | 60.2 Hz
t : 997 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                              

In [37]:
sample, sample.trajectories.shape

(Sample(trajectories=tensor([[[ 0.0337,  0.0111,  0.8242,  ..., -0.6048, -0.5567, -0.1129],
          [ 0.0170,  0.0163,  0.8170,  ..., -0.5990, -0.5310, -0.1377],
          [-0.0095,  0.0053,  0.8135,  ..., -0.6063, -0.5460, -0.1632],
          ...,
          [ 0.2703, -0.0321,  0.8527,  ..., -0.2941, -0.6261, -0.0925],
          [ 0.2802, -0.0190,  0.8154,  ..., -0.2590, -0.5559, -0.1036],
          [ 0.3000, -0.0218,  0.8314,  ..., -0.3109, -0.5209, -0.1182]]],
        device='cuda:0'), values=tensor([0.], device='cuda:0'), chains=None),
 torch.Size([1, 64, 35]))

In [38]:
dataset[0], dataset[0].trajectories.shape

(Batch(trajectories=tensor([[ 0.0000,  0.0000,  0.8251,  ..., -0.5877, -0.5578, -0.1123],
         [-0.0098, -0.0053,  0.8223,  ..., -0.6034, -0.5496, -0.1377],
         [-0.0205, -0.0135,  0.8187,  ..., -0.6073, -0.5372, -0.1672],
         ...,
         [ 0.2570, -0.0458,  0.8492,  ..., -0.3039, -0.5771, -0.0356],
         [ 0.2659, -0.0516,  0.8421,  ..., -0.2956, -0.5725, -0.0793],
         [ 0.2787, -0.0612,  0.8371,  ..., -0.3081, -0.5633, -0.1231]]), conditions={0: tensor([ 0.0000,  0.0000,  0.8251, -0.9969,  0.0457, -0.0639, -0.0029, -0.0399,
          0.0621, -0.0921,  0.0930,  0.4230,  0.1155, -1.4425,  0.4709, -0.4550,
          1.1228,  1.3071, -0.7625, -0.2475,  1.4452, -0.1059, -0.4803,  0.0466,
         -0.8114,  0.0888, -0.3455, -0.1734,  0.2040, -0.1554,  0.3841, -0.6268,
         -0.5877, -0.5578, -0.1123])}),
 torch.Size([64, 35]))

In [39]:
import numpy as np

def save_motions(sample, output_dir, filename="motion.npy"):
    filepath = os.path.join(output_dir, filename)
    pos_data = sample.trajectories.squeeze(0).cpu().numpy()
    np.save(filepath, pos_data)
    print(f"Motion {i} saved as {filename}")

save_motions(sample, f"/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser/sampled_motions", filename="motion1.npy")

Motion 0 saved as motion1.npy
